# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Mollière  ->  P. Mollière  |  ['P. Mollière']
F. Walter  ->  F. Walter  |  ['F. Walter']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
A. Feldmeier-Krause  ->  A. Feldmeier-Krause  |  ['A. Feldmeier-Krause']


A. Hughes  ->  A. Hughes  |  ['A. Hughes']
Arxiv has 59 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.02488


extracting tarball to tmp_2309.02488...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

 done.


N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
A. Feldmeier-Krause  ->  A. Feldmeier-Krause  |  ['A. Feldmeier-Krause']


Found 94 bibliographic references in tmp_2309.02503/paper.bbl.
syntax error in line 543: '=' expected
Retrieving document from  https://arxiv.org/e-print/2309.03186


extracting tarball to tmp_2309.03186...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.02488-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.02488) | **ELemental abundances of Planets and brown dwarfs Imaged around Stars  (ELPIS): I. Potential Metal Enrichment of the Exoplanet AF Lep b and a Novel  Retrieval Approach for Cloudy Self-luminous Atmospheres**  |
|| Z. Zhang, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2023-09-07*|
|*Comments*| *AJ, in press. Main text: Pages 1-32, Figures 1-15, Tables 1-6. All figures and tables after References belong to the Appendix (Pages 32-58, Figures 16-20, Table 7). For supplementary materials, please refer to the Zenodo repository this https URL*|
|**Abstract**| AF Lep A+b is a remarkable planetary system hosting a gas-giant planet that has the lowest dynamical mass among directly imaged exoplanets. We present an in-depth analysis of the atmospheric composition of the star and planet to probe the planet's formation pathway. Based on new high-resolution spectroscopy of AF Lep A, we measure a uniform set of stellar parameters and elemental abundances (e.g., [Fe/H] = $-0.27 \pm 0.31$ dex). The planet's dynamical mass ($2.8^{+0.6}_{-0.5}$ M$_{\rm Jup}$) and orbit are also refined using published radial velocities, relative astrometry, and absolute astrometry. We use petitRADTRANS to perform chemically-consistent atmospheric retrievals for AF Lep b. The radiative-convective equilibrium temperature profiles are incorporated as parameterized priors on the planet's thermal structure, leading to a robust characterization for cloudy self-luminous atmospheres. This novel approach is enabled by constraining the temperature-pressure profiles via the temperature gradient $(d\ln{T}/d\ln{P})$, a departure from previous studies that solely modeled the temperature. Through multiple retrievals performed on different portions of the $0.9-4.2$ $\mu$m spectrophotometry, along with different priors on the planet's mass and radius, we infer that AF Lep b likely possesses a metal-enriched atmosphere ([Fe/H] $> 1.0$ dex). AF Lep b's potential metal enrichment may be due to planetesimal accretion, giant impacts, and/or core erosion. The first process coincides with the debris disk in the system, which could be dynamically excited by AF Lep b and lead to planetesimal bombardment. Our analysis also determines $T_{\rm eff} \approx 800$ K, $\log{(g)} \approx 3.7$ dex, and the presence of silicate clouds and dis-equilibrium chemistry in the atmosphere. Straddling the L/T transition, AF Lep b is thus far the coldest exoplanet with suggested evidence of silicate clouds. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.02493-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.02493) | **NOEMA observations of GN-z11: Constraining Neutral Interstellar Medium  and Dust Formation in the Heart of Cosmic Reionization at $z=10.6$**  |
|| Y. Fudamoto, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2023-09-07*|
|*Comments*| *submitted to MNRAS, 7 pages, 4 figures, 1 table*|
|**Abstract**| We present results of dust continuum and [CII]$\,158\,{\rm \mu m}$ emission line observations of a remarkably UV-luminous ($M_{\rm UV}=-21.6$) galaxy at $z=10.603$: GN-z11. Using the Northern Extended Millimeter Array (NOEMA), observations have been carried out over multiple observing cycles. We achieved a high sensitivity resulting in a $\lambda_{\rm rest}=160\,{\rm \mu m}$ continuum $1\,\sigma$ depth of $13.0\,\rm{\mu Jy/beam}$ and a [CII] emission line $1\,\sigma$ sensitivity of $31\,\rm{mJy/beam\,km/s}$ using $50\,\rm{km/s}$ binning with a $\sim 2\,{\rm arcsec}$ synthesized beam. Neither dust continuum nor [CII]$\,158\,{\rm \mu m}$ line emission are detected at the expected frequency of $\nu_{\rm [CII]} = 163.791\,\rm{GHz}$ and the sky location of GN-z11. The upper limits show that GN-z11 is neither luminous in $L_{\rm IR}$ nor $L_{\rm [CII]}$, with a dust mass $3\,\sigma$ limit of ${\rm log}(M_{\rm dust}/{\rm M_{\odot}}) < 6.5-6.9$ and with a [CII] based molecular gas mass $3\,\sigma$ limit of ${\rm log}(M_{\rm mol,[CII]}/{\rm M_{\odot}}) < 9.3$. Together with radiative transfer calculations, we also investigated the possible cause of the dust poor nature of the GN-z11 showed by the blue color in the UV continuum of GN-z11 ($\beta_{\rm UV}=-2.4$), and found that $\gtrsim3\times$ deeper observations are crucial to study dust production at very high-redshift. Nevertheless, our observations show the crucial role of deep mm/submm observations of very high redshift galaxies to constrain multiple phases in the interstellar medium. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.02503-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.02503) | **oMEGACat I: MUSE spectroscopy of 300,000 stars within the half-light  radius of $ω$ Centauri**  |
|| M. S. Nitschai, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>, <mark>M. Häberle</mark>, <mark>A. Feldmeier-Krause</mark> |
|*Appeared on*| *2023-09-07*|
|*Comments*| *27 pages, 18 figures, 3 tables, accepted for publication in ApJ, the catalog will be available in the online material of the published article*|
|**Abstract**| Omega Centauri ($\omega$ Cen) is the most massive globular cluster of the Milky Way and has been the focus of many studies that reveal the complexity of its stellar populations and kinematics. However, most previous studies have used photometric and spectroscopic datasets with limited spatial or magnitude coverage, while we aim to investigate it having full spatial coverage out to its half-light radius and stars ranging from the main sequence to the tip of the red giant branch. This is the first paper in a new survey of $\omega$ Cen that combines uniform imaging and spectroscopic data out to its half-light radius to study its stellar populations, kinematics, and formation history. In this paper, we present an unprecedented MUSE spectroscopic dataset combining 87 new MUSE pointings with previous observations collected from guaranteed time observations. We extract spectra of more than 300,000 stars reaching more than two magnitudes below the main sequence turn-off. We use these spectra to derive metallicity and line-of-sight velocity measurements and determine robust uncertainties on these quantities using repeat measurements. Applying quality cuts we achieve signal-to-noise ratios of 16.47/73.51 and mean metallicity errors of 0.174/0.031 dex for the main sequence stars (18 mag $\rm < mag_{F625W}<$22 mag) and red giant branch stars (16 mag $<\rm mag_{F625W}<$10 mag), respectively. We correct the metallicities for atomic diffusion and identify foreground stars. This massive spectroscopic dataset will enable future studies that will transform our understanding of $\omega$ Cen, allowing us to investigate the stellar populations, ages, and kinematics in great detail. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.03186-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.03186) | **HCN emission from translucent gas and UV-illuminated cloud edges  revealed by wide-field IRAM 30m maps of Orion B GMC: Revisiting its role as  tracer of the dense gas reservoir for star formation**  |
|| M. G. Santa-Maria, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-09-07*|
|*Comments*| *accepted for publication in A&A. 24 pages, 18 figures, plus Appendix. Abridged Abstract. English language not edited*|
|**Abstract**| We present 5 deg^2 (~250 pc^2) HCN, HNC, HCO+, and CO J=1-0 maps of the Orion B GMC, complemented with existing wide-field [CI] 492 GHz maps, as well as new pointed observations of rotationally excited HCN, HNC, H13CN, and HN13C lines. We detect anomalous HCN J=1-0 hyperfine structure line emission almost everywhere in the cloud. About 70% of the total HCN J=1-0 luminosity arises from gas at A_V < 8 mag. The HCN/CO J=1-0 line intensity ratio shows a bimodal behavior with an inflection point at A_V < 3 mag typical of translucent gas and UV-illuminated cloud edges. We find that most of the HCN J=1-0 emission arises from extended gas with n(H2) < 10^4 cm^-3, even lower density gas if the ionization fraction is > 10^-5 and electron excitation dominates. This result explains the low-A_V branch of the HCN/CO J=1-0 intensity ratio distribution. Indeed, the highest HCN/CO ratios (~0.1) at A_V < 3 mag correspond to regions of high [CI] 492 GHz/CO J=1-0 intensity ratios (>1) characteristic of low-density PDRs. Enhanced FUV radiation favors the formation and excitation of HCN on large scales, not only in dense star-forming clumps. The low surface brightness HCN and HCO+ J=1-0 emission scale with I_FIR (a proxy of the stellar FUV radiation field) in a similar way. Together with CO J=1-0, these lines respond to increasing I_FIR up to G0~20. On the other hand, the bright HCN J=1-0 emission from dense gas in star-forming clumps weakly responds to I_FIR once the FUV radiation field becomes too intense (G0>1500). The different power law scalings (produced by different chemistries, densities, and line excitation regimes) in a single but spatially resolved GMC resemble the variety of Kennicutt-Schmidt law indexes found in galaxy averages. As a corollary for extragalactic studies, we conclude that high HCN/CO J=1-0 line intensity ratios do not always imply the presence of dense gas. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.02488.md
    + _build/html/tmp_2309.02488/./assist_violin_key_parameters.png
    + _build/html/tmp_2309.02488/./AFLepb_spectrophotometry.png
    + _build/html/tmp_2309.02488/./Mesa23DeRosa23spec_flex_offsets_allphot_massrad_PTslope_retrieval_results.png
exported in  _build/html/2309.02493.md
    + _build/html/tmp_2309.02493/./GNz11_c2_mom0.v3.png
    + _build/html/tmp_2309.02493/./GNz11_continuum.v0.png
    + _build/html/tmp_2309.02493/./SFR-LC2.v1_mod.png
exported in  _build/html/2309.02503.md
    + _build/html/tmp_2309.02503/./figures/Figure5.png
    + _build/html/tmp_2309.02503/./figures/Figure2.png
    + _build/html/tmp_2309.02503/./figures/Figure8a.png
    + _build/html/tmp_2309.02503/./figures/Figure8b.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\dotdeg}{\rlap{.}^\circ}$
$\newcommand{\dotarcsec}{\rlap{.}"}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>



<div id="title">

# ELemental abundances of Planets and brown dwarfs Imaged around Stars (ELPIS): I. Potential Metal Enrichment of the Exoplanet AF Lep b and a Novel Retrieval Approach for Cloudy Self-luminous Atmospheres

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.02488-b31b1b.svg)](https://arxiv.org/abs/2309.02488)<mark>Appeared on: 2023-09-07</mark> -  _AJ, in press. Main text: Pages 1-32, Figures 1-15, Tables 1-6. All figures and tables after References belong to the Appendix (Pages 32-58, Figures 16-20, Table 7). For supplementary materials, please refer to the Zenodo repository this https URL_

</div>
<div id="authors">

Z. Z. (张周健), et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** AF Lep A+b is a remarkable planetary system hosting a gas-giant planet that has the lowest dynamical mass among directly imaged exoplanets. We present an in-depth analysis of the atmospheric composition of the star and planet to probe the planet's formation pathway. Based on new high-resolution spectroscopy of AF Lep A, we measure a uniform set of stellar parameters and elemental abundances (e.g., [ Fe/H ] $=-0.27 \pm 0.31$ dex). The planet's dynamical mass ( $2.8^{+0.6}_{-0.5}$ M $_{\rm Jup}$ ) and orbit are also refined using published radial velocities, relative astrometry, and absolute astrometry. We use \texttt{petitRADTRANS} to perform chemically-consistent atmospheric retrievals for AF Lep b. The radiative-convective equilibrium temperature profiles are incorporated as parameterized priors on the planet's thermal structure, leading to a robust characterization for cloudy self-luminous atmospheres. This novel approach is enabled by constraining the temperature-pressure profiles via the temperature gradient $(d\ln{T}/d\ln{P})$ , a departure from previous studies that solely modeled the temperature. Through multiple retrievals performed on different portions of the $0.9-4.2$  $\mu$ m spectrophotometry, along with different priors on the planet's mass and radius, we infer that AF Lep b likely possesses a metal-enriched atmosphere ( [ Fe/H ] $>1.0$ dex). AF Lep b's potential metal enrichment may be due to planetesimal accretion, giant impacts, and/or core erosion. The first process coincides with the debris disk in the system, which could be dynamically excited by AF Lep b and lead to planetesimal bombardment. Our analysis also determines $T_{\rm eff} \approx 800$ K, $\log{(g)} \approx 3.7$ dex, and the presence of silicate clouds and dis-equilibrium chemistry in the atmosphere. Straddling the L/T transition, AF Lep b is thus far the coldest exoplanet with suggested evidence of silicate clouds.

</div>

<div id="div_fig1">

<img src="tmp_2309.02488/./assist_violin_key_parameters.png" alt="Fig25" width="100%"/>

**Figure 25. -** Violin plot for the posteriors of several key physical and chemical parameters derived from all the nine retrieval runs (Section \ref{sec:retrieval}). The left three columns (blue) correspond to the retrievals on the $K1/K2/L'$ photometry and the \cite{2023AandA...672A..93M} spectrum (Section \ref{subsec:phot_m23spec}). The middle three columns (green) correspond to the retrievals on the $K1/K2/L'$ photometry and the \cite{2023AandA...672A..94D} spectrum (Section \ref{subsec:phot_d23spec}). The right three columns (red) correspond to the retrievals on the $K1/K2/L'$ photometry and both \cite{2023AandA...672A..93M} and \cite{2023AandA...672A..94D} spectra. Plots with darker blue/green/red colors suggest the addition of narrower and constrained priors on $M$(slightly darker) or both $M$ and $R$(much darker). The measured [Fe/H] of the host star AF Lep A ($-0.27 \pm 0.31$ dex; Section \ref{sec:host_param_abund}) is shown as the purple shade in the top panel.  (*fig:violin_key*)

</div>
<div id="div_fig2">

<img src="tmp_2309.02488/./AFLepb_spectrophotometry.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spectrophotometry of AF Lep b from \citeauthor{2023AandA...672A..93M}(\citeyear{2023AandA...672A..93M}; blue), \citeauthor{2023AandA...672A..94D}(\citeyear{2023AandA...672A..94D}; green), and \citeauthor{2023ApJ...950L..19F}(\citeyear{2023ApJ...950L..19F}; purple). Photometry is converted from magnitudes into fluxes (unless reported in the literature) based on zero points provided by \cite{2017AJ....154..218N}. Response curves of $K1/K2/L'$ bands (grey) are obtained from the VLT/SPHERE and Keck/NIRC2 websites.  (*fig:specphot*)

</div>
<div id="div_fig3">

<img src="tmp_2309.02488/./Mesa23DeRosa23spec_flex_offsets_allphot_massrad_PTslope_retrieval_results.png" alt="Fig23" width="100%"/>

**Figure 23. -** Results of the retrieval analysis on $K1/K2/L'$ photometry and both \cite{2023AandA...672A..93M} and \cite{2023AandA...672A..94D} spectra of AF Lep b (Section \ref{subsec:phot_d23spec}), with the same format as Figure \ref{fig:results_m23}. (*fig:results_m23d23*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.02488"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\magenta}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\lfir}{L_{\rm FIR}}$
$\newcommand{\lfuv}{L_{\rm FUV}}$
$\newcommand{\lcii}{L_{\rm[CII]}}$
$\newcommand{\luv}{L_{\rm UV}}$
$\newcommand{\lir}{L_{\rm IR}}$
$\newcommand{\lsol}{L_{\odot}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\sfrlya}{SFR_{Ly\alpha}}$
$\newcommand{\sfrha}{SFR_{H\alpha}}$
$\newcommand{\sfruv}{SFR_{UV}}$
$\newcommand{\sfrir}{SFR_{IR}}$
$\newcommand{\sfrtot}{SFR_{TOT}}$
$\newcommand{\msolyr}{M_{\odot}~yr^{-1}}$
$\newcommand{\micron}{\mum}$
$\newcommand{\hyperz}{{\em Hyperz}}$
$\newcommand{\myr}{M_{\odot}~yr^{-1}}$
$\newcommand{\msunyr}{M_{\odot}~yr^{-1}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\zphot}{\ifmmode z_{\rm phot}\elsez_{\rm phot}\fi}$
$\newcommand{\sskip}{\vskip 3truept \noindent}$
$\newcommand{\Oiii}{[O {\sc iii}]}$
$\newcommand{\Neiii}{[Ne {\sc iii}]}$
$\newcommand{\Oii}{[O {\sc ii}]}$
$\newcommand{\Cii}{[C {\sc ii}]}$
$\newcommand{\Nii}{[N {\sc ii}]}$
$\newcommand{\Oi}{[O {\sc i}]}$
$\newcommand{\ltsima}{\buildrel<\over\sim}$
$\newcommand{\la}{\lower.5ex\hbox{\ltsima}~}$
$\newcommand{\gtsima}{\buildrel>\over\sim}$
$\newcommand{\ga}{\lower.5ex\hbox{\gtsima}~}$
$\newcommand{\}{deg}$</div>



<div id="title">

# NOEMA observations of GN-z11: Constraining Neutral Interstellar Medium and Dust Formation in the Heart of Cosmic Reionization at $z=10.6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.02493-b31b1b.svg)](https://arxiv.org/abs/2309.02493)<mark>Appeared on: 2023-09-07</mark> -  _submitted to MNRAS, 7 pages, 4 figures, 1 table_

</div>
<div id="authors">

Y. Fudamoto, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present results of dust continuum and $\Cii$ $ 158 {\rm \mu m}$ emission line observations of a remarkably UV-luminous ( $M_{\rm UV}=-21.6$ ) galaxy at $z=10.603$ : GN-z11.Using the Northern Extended Millimeter Array (NOEMA),  observations have been carried out over multiple observing cycles. We achieved a high sensitivity resulting in a $\lambda_{\rm rest}=160 {\rm \mu m}$ continuum $1 \sigma$ sensitivity of $13.0 \rm{\mu Jy/beam}$ and a $\Cii$ emission line $1 \sigma$ sensitivity of $31 \rm{mJy/beam km/s}$ using $50 \rm{km/s}$ binning with a $\sim 2 {\rm arcsec}$ synthesized beam.Neither dust continuum nor $\Cii$ $ 158 {\rm \mu m}$ line emission are detected at the expected frequency of $\nu_{\rm[CII]} = 163.791 \rm{GHz}$ and the sky location of GN-z11.The upper limits show that GN-z11 is neither luminous in $\lir$ nor $L_{\rm[CII]}$ , with a dust mass $3 \sigma$ limit of ${\rm log}(M_{\rm dust}/{\rm M_{\odot}}) < 6.5-6.9$ and with a $\Cii$ based molecular gas mass $3 \sigma$ limit of ${\rm log}(M_{\rm mol,[CII]}/{\rm M_{\odot}}) < 9.3$ .Together with radiative transfer calculations, we also investigated the possible cause of the dust poor nature of the GN-z11 showed by the blue color in the UV continuum of GN-z11 ( $\beta_{\rm UV}=-2.4$ ), and found that $\gtrsim3\times$ deeper observations are crucial to study dust production at very high-redshift.Nevertheless, our observations show the crucial role of deep mm/submm observations of very high redshift galaxies to constrain multiple phases in the interstellar medium.

</div>

<div id="div_fig1">

<img src="tmp_2309.02493/./GNz11_c2_mom0.v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** ** Left Panel: ** The NOEMA spectrum of GN-z11 with $50 \mathrm{km/s}$ binning. The gray solid line shows the RMS of each channel. The NOEMA observation covers $\Cii$$ 158 \rm{\mu m}$ emission line of GN-z11 at the observed frequency of $\nu_{\rm obs}=163.84 {\rm GHz}$ with the $z_{\rm spec}=10.60$(Red line). From the data cube and extracted spectrum, we do not find any signal of the $\Cii$ emission line.
    ** Right panel:**$8^{\prime\prime}\times8^{\prime\prime}$ cutout of JWST F444W image (background) and $\Cii$ emission line moment-0 map (contours) of GN-z11. Contours show 1, 2, 3$\sigma$ and dashed contours show -3, -2, -1$\sigma$. The moment-0 map of the data cube is made by integrating over the $150 \rm{km/s}$ of the $\Cii$ emission line frequency. RMS of the image is $29 {\rm mJy km/s}$, providing a $3 \sigma$ upper limit of $\Cii$ luminosity $<1.7\times10^{8} {\rm L_{\odot}}$.
     (*fig:c2*)

</div>
<div id="div_fig2">

<img src="tmp_2309.02493/./GNz11_continuum.v0.png" alt="Fig1" width="100%"/>

**Figure 1. -** $8^{\prime\prime}\times8^{\prime\prime}$ cutout of the JWST F444W image obtained by the FRESCO JWST survey (background;  ([ and Oesch 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230402026O)) ) and dust continuum (orange contours) of GN-z11: Solid contours show 1, 2$ \sigma$ and dashed contours show -3, -2, -1$\sigma$ where $1 \sigma=13.0 \mathrm{\mu Jy/beam}$. The white ellipse in the lower left corner shows the synthesized beam FWHM of the combined dust continuum image ($2.1^{\prime\prime}\times1.8^{\prime\prime}$).
     (*fig:continuum*)

</div>
<div id="div_fig3">

<img src="tmp_2309.02493/./SFR-LC2.v1_mod.png" alt="Fig2" width="100%"/>

**Figure 2. -** Star formation rate versus $\Cii$ emission line luminosity of GN-z11. Previous observations of $z>6$ are also plotted with gray points ( ([ and Harikane 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...896...93H)) ;  ([ and Fudamoto 2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230307513F)) ;  ([ and Schouws 2022](https://ui.adsabs.harvard.edu/abs/2022arXiv220204080S))  and references therein). Downward triangles show $3 \sigma$ upper limit in case of non-detections. Lines show previously obtained relations for low- and high-redshift galaxies (solid:  ([ and Schaerer 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...643A...3S)) ).
    For the SFR of GN-z11, we adopted the SED fitting results of $21^{+22}_{-10} {\rm M_{\odot} yr^{-1}}$ from [ and Tacchella (2023)](https://ui.adsabs.harvard.edu/abs/2023arXiv230207234T). (*fig:C2SFR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.02493"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$
$\newcommand{\figureautorefname}{Fig.}$</div>



<div id="title">

# oMEGACat I: MUSE spectroscopy of 300,000 stars within the half-light radius of $\omega$ Centauri

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.02503-b31b1b.svg)](https://arxiv.org/abs/2309.02503)<mark>Appeared on: 2023-09-07</mark> -  _27 pages, 18 figures, 3 tables, accepted for publication in ApJ, the catalog will be available in the online material of the published article_

</div>
<div id="authors">

M. S. Nitschai, et al. -- incl., <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>, <mark>M. Häberle</mark>, <mark>A. Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** Omega Centauri ( $\omega$ Cen) is the most massive globular cluster of the Milky Way and has been the focus of many studies that reveal the complexity of its stellar populations and kinematics. However, most previous studies have used photometric and spectroscopic datasets with limited spatial or magnitude coverage, while we aim to investigate it having full spatial coverage out to its half-light radius and stars ranging from the main sequence to the tip of the red giant branch. This is the first paper in a new survey of $\omega$ Cen that combines uniform imaging and spectroscopic data out to its half-light radius to study its stellar populations, kinematics, and formation history. In this paper, we present an unprecedented MUSE spectroscopic dataset combining 87 new MUSE pointings with previous observations collected from guaranteed time observations. We extract spectra of more than 300,000 stars reaching more than two magnitudes below the main sequence turn-off. We use these spectra to derive metallicity and line-of-sight velocity measurements and determine robust uncertainties on these quantities using repeat measurements. Applying quality cuts we achieve signal-to-noise ratios of 16.47/73.51 and mean metallicity errors of 0.174/0.031 dex for the main sequence stars ( $\SI{18}{\mag}$ $\rm < mag_{F625W}<$ $\SI{22}{\mag}$ ) and red giant branch stars ( $\SI{16}{\mag}$ $<\rm  mag_{F625W}<$ $\SI{10}{\mag}$ ), respectively. We correct the metallicities for atomic diffusion and identify foreground stars. This massive spectroscopic dataset will enable future studies that will transform our understanding of $\omega$ Cen, allowing us to investigate the stellar populations, ages, and kinematics in great detail.

</div>

<div id="div_fig1">

<img src="tmp_2309.02503/./figures/Figure5.png" alt="Fig2" width="100%"/>

**Figure 2. -** **CMD of GO stars.** CMD of the stars in the GO dataset. The black dots show the stars with fixed log(g) in our spexxy fit while the orange points have free log(g) and the purple are the HB stars which are disregarded in the current analysis. The yellow points indicate the position of the isochrone used to infer the log(g).  (*fig:CMD spexxy*)

</div>
<div id="div_fig2">

<img src="tmp_2309.02503/./figures/Figure2.png" alt="Fig12" width="100%"/>

**Figure 12. -** **Image of $\omega$ Cen and footprint of the MUSE pointings.** On the left is a grayscale image created from the combined GO and GTO MUSE WFM data. Overlaid blue colored squares indicate the GO AO pointings, red squares the WFM GTO data, while the region without squares is where the non-AO GO WFM pointings lie. On the right is the footprint of the individual pointings showing the overlap of the data. At the center of the right-hand Figure, the NFM data are within the yellow region at the center of the image, indicated with a red cross.
 (*fig:data*)

</div>
<div id="div_fig3">

<img src="tmp_2309.02503/./figures/Figure8a.png" alt="Fig15.1" width="50%"/><img src="tmp_2309.02503/./figures/Figure8b.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** **Error analysis correction.** Top panels: The x-axis is the SNR and the y-axis is the $\sigma^2$ of the $\Delta[M/H]$ for the GO data on the left and the GTO data on the right. The red line is the best-fitting first-order polynomial, the black dots are our data values, and the light red shaded area is the region below an SNR of 10, which we exclude from the analysis. Bottom panels: The x-axis is the SNR and the maximum percentage of counts one star has in each bin. The red dashed line shows the maximum number of measurements each star can have, which is clearly always below 10 \% of the total number of measurements in each bin, making sure the statistics in each bin are not the result of only one star. (*fig:error SN*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.02503"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

264  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
